In [6]:
import torch
import torchvision.transforms as transforms

from easydict import EasyDict as edict

from recognition.arcface_torch.backbones import get_model
from recognition.arcface_torch.configs.aihub_r50_onegpu import config as cfg

from datasets.AIHubDataset import AIHubDataset
from validate_aihub import validate_aihub

In [29]:
TASK = "Family"
SPLIT = "test"
aihub_dataroot = f"/home/jupyter/data/face-image/{SPLIT}_aihub_family"
num_workers = 4
lfw_batch_size = 200
image_size = 112
aihub_mean = [0.5444, 0.4335, 0.3800]
aihub_std = [0.2672, 0.2295, 0.2156]

In [8]:
aihub_transforms = transforms.Compose(
    [
        transforms.Resize(size=(image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)


In [30]:
aihub_dataloader = torch.utils.data.DataLoader(
    dataset=AIHubDataset(
        dir=aihub_dataroot,
        pairs_path=f"data/pairs/{SPLIT}/pairs_{TASK}.txt",
        transform=aihub_transforms,
    ),
    batch_size=lfw_batch_size,
    num_workers=num_workers,
    shuffle=False,
)


In [10]:
backbone = get_model(
    cfg.network, dropout=0.0, fp16=cfg.fp16, num_features=cfg.embedding_size
)

In [32]:
model_weights = torch.load(
    f"/home/jupyter/face/utils/model/arcface/{cfg.network}/backbone.pth"
)
backbone.load_state_dict(model_weights)

<All keys matched successfully>

### fine-tune weight

In [33]:
target_weight_path = "/home/jongphago/insightface/work_dirs/aihub_r50_onegpu/checkpoint_gpu_0.pt"
state_dict = torch.load(target_weight_path)
backbone.load_state_dict(state_dict['state_dict_backbone'])

<All keys matched successfully>

In [34]:
backbone.cuda().eval()
_ = validate_aihub(backbone, aihub_dataloader, cfg.network, 0)

Validating on AIHUB! ...


100%|██████████| 30/30 [00:31<00:00,  1.04s/it]


Accuracy on AIHUB: 0.9900+-0.0025	Precision 0.9930+-0.0038	Recall 0.9870+-0.0043	ROC Area Under Curve: 0.6229	Best distance threshold: 21.41+-0.18	TAR: 0.7103+-0.0396 @ FAR: 0.0013
Accuracy on AIHUB, Precision, Recall, ROC Area Under Curve, Best distance threshold TAR, FAR
0.9900+-0.0025, 0.9930+-0.0038, 0.9870+-0.0043,0.6229, 21.41+-0.18,0.7103+-0.0396, 0.0013
